# Breast cancer biopsy diagnosis using SVM classifier

Here we attempt to use a support vector machine to predict biopsy classifications using the UCI Wisconsin breast cancer database dataset.  We optimise SVM parameters using Scikit Learn's GridSearch method.

Datasset URL: https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)
(dataset also abailable via sklearn).

The dataset contains labelled data: Diagnosis = B for benign and M for malignant 10.

The UCI website describes it as containing 10 real-valued features relating to cell nuclei obtained from the biopsy:

    a) radius (mean of distances from center to points on the perimeter) 
    b) texture (standard deviation of gray-scale values) 
    c) perimeter 
    d) area 
    e) smoothness (local variation in radius lengths) 
    f) compactness (perimeter^2 / area - 1.0) 
    g) concavity (severity of concave portions of the contour) 
    h) concave points (number of concave portions of the contour) 
    i) symmetry 
    j) fractal dimension ("coastline approximation" - 1)
    
Further derived features (mean, standard error, and "worst" i.e. mean of the three largest values) are obtained from the above giving a total of 30 features.

## 1) Import libraries

In [108]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## 2) Get data

In [109]:
from sklearn.datasets import load_breast_cancer

In [110]:
cancer = load_breast_cancer()

## 3) Exploratory data analysis

In [111]:
type(cancer)

sklearn.datasets.base.Bunch

From the above we see that the sklearn version of the dataset exists as a base.Bunch - a 'dictionary-like' format.
We can see parent level elements using .keys()... 

In [112]:
cancer.keys()

dict_keys(['feature_names', 'target_names', 'target', 'DESCR', 'data'])

We can access the keys using typical pandas referencing...

In [113]:
cancer['target_names']

array(['malignant', 'benign'], 
      dtype='<U9')

, or using shortcut (if no spaces present)...

In [114]:
cancer.target_names

array(['malignant', 'benign'], 
      dtype='<U9')

Let's look at the dataset by means of the keys...

In [116]:
cancer.keys() 

dict_keys(['feature_names', 'target_names', 'target', 'DESCR', 'data'])

We wrap each key with print() function to display info nicely...

In [117]:
print(cancer['DESCR'])

Breast Cancer Wisconsin (Diagnostic) Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, field
        13 is Radius SE, field 23 is Worst Radius.

        

In [118]:
print(cancer['data'])

[[  1.79900000e+01   1.03800000e+01   1.22800000e+02 ...,   2.65400000e-01
    4.60100000e-01   1.18900000e-01]
 [  2.05700000e+01   1.77700000e+01   1.32900000e+02 ...,   1.86000000e-01
    2.75000000e-01   8.90200000e-02]
 [  1.96900000e+01   2.12500000e+01   1.30000000e+02 ...,   2.43000000e-01
    3.61300000e-01   8.75800000e-02]
 ..., 
 [  1.66000000e+01   2.80800000e+01   1.08300000e+02 ...,   1.41800000e-01
    2.21800000e-01   7.82000000e-02]
 [  2.06000000e+01   2.93300000e+01   1.40100000e+02 ...,   2.65000000e-01
    4.08700000e-01   1.24000000e-01]
 [  7.76000000e+00   2.45400000e+01   4.79200000e+01 ...,   0.00000000e+00
    2.87100000e-01   7.03900000e-02]]


In [119]:
print(cancer['feature_names'])

['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']


In [120]:
print(cancer['target'])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1
 1 1 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0
 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 0 1 1
 0 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 0 1 0 1 1 0 

In [121]:
print(cancer['target_names'])

['malignant' 'benign']


From the above we would assume that malignant = 0 and benign = 1.  (This info isn't explicitly stated in dataset description.)  Let's verify.  We know from description that there are that there are 357 benign cases.  If benign =1, then summing the target variables will give us 357.  Let's confirm this is the case...

In [122]:
sum(cancer.target)

357

Let's create a dataframe for the dataset feature variables (leaving out the target data) ...

In [123]:
df = pd.DataFrame(cancer['data'], columns=cancer['feature_names']) 

In [124]:
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [125]:
df.columns

Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension'],
      dtype='object')

In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
mean radius                569 non-null float64
mean texture               569 non-null float64
mean perimeter             569 non-null float64
mean area                  569 non-null float64
mean smoothness            569 non-null float64
mean compactness           569 non-null float64
mean concavity             569 non-null float64
mean concave points        569 non-null float64
mean symmetry              569 non-null float64
mean fractal dimension     569 non-null float64
radius error               569 non-null float64
texture error              569 non-null float64
perimeter error            569 non-null float64
area error                 569 non-null float64
smoothness error           569 non-null float64
compactness error          569 non-null float64
concavity error            569 non-null float64
concave points error       569 non-null float64
symmetry error             569 

From the above, we can verify: 

    There are 30 features.
    All are floats.
    There are no missing values.
    There are no NaNs.

In [127]:
df.describe()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


Looking at the above, we might ask if some of these features should be optimised to help the algorithm converge.  However for now, we'll go with the simplest approach first and just use the data 'as is'.

## 4) Create model 

Let's create our X matrix and y target data...

In [128]:
X = df # We know this is a dataframe
y = cancer.target # Let's see what type this is
type(y)

numpy.ndarray

Split data into training and test data...

In [129]:
from sklearn.model_selection import train_test_split

In [130]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

Import support vector classifier class...

In [131]:
from sklearn.svm import SVC

Instantiantiate classifier object...

In [132]:
classifier = SVC()

### 5) Evaluate model

Fit training data...

In [133]:
classifier.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Make predictions using test set...

In [134]:
y_pred = classifier.predict(X_test)

Assess performance...

In [135]:
from sklearn.metrics import confusion_matrix, classification_report

In [136]:
print(confusion_matrix(y_test, y_pred))

[[  0  66]
 [  0 105]]


From the confusion matrix, we note the following:

    There are 66 actual class 0 (malignant) 
    There are 105 actual class 1 (benign)
    
It seems that our SVM is classifying everything as benign.  All actual class 0 (malignant) are coming out as false positives.  Let's look at the precision and recall...

In [137]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        66
          1       0.61      1.00      0.76       105

avg / total       0.38      0.61      0.47       171



/home/fluff/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


As expected, for class 1 (benign) recall = 1 because the SVM is finding all the benign cases, however it is also pulling in _all_ malignant cases so it's precision for this class is poor.

Recall for class 0 (malignant) is zero because the SVM is unable to find these cases 

Obviously this is a very serious failing for a cancer detection system.  Where positive corresponds to benign, we would prefer false negatives (incorrectly diagnosing benign as malignant) rather than false positives (incorrectly classifying malignant cancers as benign - i.e. failing to spot malignant cancer)



We need to adjust the parameters of the SVM (and perhaps consider normalising data).


Let's search for best parameters using sklearn's GridSearch

In [138]:
from sklearn.model_selection import GridSearchCV

GridSearchCV takes in dict (or list of dicts) 
the dict contains parameter names as keys and parameter settings as values
GridSearch then uses these various hyperparameter values to train multiple SVMs

So...
keys = params, values = list of settings to test.

We know the parameters from looking at help for our instantiation call: classifier = SVC() 

Here we will try a Grid based on a range of C and gamma values.

C in SVM is the penalty for misclassifying. High C value corresponds to strict margin.  Relaxing may help. <br>
**Large C = low bias, high variance** ,so model is strictly trained to training data - overfitted - and can't generalise to test data).

Gamma is the kernel coefficient (or it is when kernel is default = radial basis function).  It defines how far the influence a single training example reaches.  Intuitively, if gamma is high then each example exerts far recahing influence and gives potential for overfittig (high variance)  Thus gamma works in reverse way to C: <br> 
**Large gamma = High bias** , biased to model, rather than data, and thus low variance (low overfitting). 

Let's define a parameter grid to explore the function of the SVM given a range of C and gamma parameters...

In [139]:
param_grid ={'C':[0.1, 1, 10, 100, 1000],
            'gamma':[1, 0.1, 0.01, 0.001, 0.0001]}

Instantiate a GridSearch object with our classifier and parameter grid...

In [140]:
grid = GridSearchCV(SVC(), param_grid, verbose=3)

Train our grid of multiple SVMs...

In [141]:
grid.fit(X_train, y_train)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] gamma=1, C=0.1 ..................................................
[CV] ................... gamma=1, C=0.1, score=0.631579, total=   0.0s
[CV] gamma=1, C=0.1 ..................................................
[CV] ................... gamma=1, C=0.1, score=0.631579, total=   0.0s
[CV] gamma=1, C=0.1 ..................................................
[CV] ................... gamma=1, C=0.1, score=0.636364, total=   0.0s
[CV] gamma=0.1, C=0.1 ................................................
[CV] ................. gamma=0.1, C=0.1, score=0.631579, total=   0.0s
[CV] gamma=0.1, C=0.1 ................................................
[CV] ................. gamma=0.1, C=0.1, score=0.631579, total=   0.0s
[CV] gamma=0.1, C=0.1 ................................................
[CV] ................. gamma=0.1, C=0.1, score=0.636364, total=   0.0s
[CV] gamma=0.01, C=0.1 ...............................................
[CV] ...........

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] ................... gamma=0.1, C=1, score=0.631579, total=   0.0s
[CV] gamma=0.1, C=1 ..................................................
[CV] ................... gamma=0.1, C=1, score=0.636364, total=   0.0s
[CV] gamma=0.01, C=1 .................................................
[CV] .................. gamma=0.01, C=1, score=0.631579, total=   0.0s
[CV] gamma=0.01, C=1 .................................................
[CV] .................. gamma=0.01, C=1, score=0.631579, total=   0.0s
[CV] gamma=0.01, C=1 .................................................
[CV] .................. gamma=0.01, C=1, score=0.636364, total=   0.0s
[CV] gamma=0.001, C=1 ................................................
[CV] ................. gamma=0.001, C=1, score=0.902256, total=   0.0s
[CV] gamma=0.001, C=1 ................................................
[CV] ................. gamma=0.001, C=1, score=0.939850, total=   0.0s
[CV] gamma=0.001, C=1 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    0.7s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'C': [0.1, 1, 10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=3)

Let's see what the grid search believes are the best hyperparameter values...

In [142]:
grid.best_params_

{'C': 10, 'gamma': 0.0001}

So this is what the classifier (estimator) looks like...

In [143]:
grid.best_estimator_

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

We can also pull out the score (gridsearch allocates a cross validation type score to each classifier in the grid)...

In [144]:
grid.best_score_

0.95477386934673369

_grid_ is used above to refer to the highest scoring classifier.  We can take _grid_ and use it as our new (tuned) classifier

## 6) Re-evaluation - based on our gridsearch tuned classifier

In [148]:
grid_pred = grid.predict(X_test)

In [149]:
print(confusion_matrix(y_test, grid_pred))

[[ 60   6]
 [  3 102]]


We see a marked improvement comparing the above confusion matrix to our previous one. 9 out of 171 samples are misclassified (~5% misclassification)

As before, we note:

    There are 66 actual class 0 (malignant)
    There are 105 actual class 1 (benign)

However now we see that our SVM correctly finds 60 of the malignant tumours, a big improvement over our earlier SVM but unfortunately it classifies 6 malignant tumours as benign (false positives).  For cancer detection, this would be a worrying performance issue.

There are 3 false negatives, which, given our labeling, corresponds to: benigns classed as malignant.  This would be acceptable in cancer detection. 

In [150]:
print( classification_report(y_test, grid_pred))

             precision    recall  f1-score   support

          0       0.95      0.91      0.93        66
          1       0.94      0.97      0.96       105

avg / total       0.95      0.95      0.95       171



For class 0 (malignant), we have a recall of 0.91.  Whilst an improvement over earlier SVM, maximising this metric would be a priority in cancer detection.  The quality of the recall is high - precision = 0.95 - only a few benigns have been classed as malignant.

For class 1 (benign), we have a recall of 0.97.  Most of the benigns are found.  Unfortunately as mentioned, we are also pulling in some malignants here hence our precision is not as high as we would like.  We would aspire to have the precision here = 1 (and recall for class 0 = 1) an aspiration which may require us to accept more benigns detected as malignant.

Further work to improve the recall for class 0 might be based on revisting the second and third best SVMs from the grid.  (The grid's scoring system may have different priorities to our requirements.)  We could also try scaling the features. 